In [39]:
#Importing necessary libraries 
import keras
import numpy as np
import pandas as pd
from keras.applications import vgg16, inception_v3, resnet50, mobilenet
from keras import models
from keras import layers
from keras import optimizers
import cv2
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import os
from matplotlib import pyplot as plt
import numpy as np
from PIL import Image

In [6]:
# Load the ResNet50 model
# Performing Fine Tuning
resnet_model = resnet50.ResNet50(weights='imagenet', include_top=False, input_shape=(64, 64, 3))

In [7]:
# checking the trainable status of the individual layers
for layer in resnet_model.layers:
    print(layer, layer.trainable)

<keras.engine.input_layer.InputLayer object at 0x7f0270882a90> False
<keras.layers.convolutional.ZeroPadding2D object at 0x7f02708686a0> True
<keras.layers.convolutional.Conv2D object at 0x7f0270868400> True
<keras.layers.normalization.BatchNormalization object at 0x7f02708647f0> True
<keras.layers.core.Activation object at 0x7f0270868358> True
<keras.layers.convolutional.ZeroPadding2D object at 0x7f0270f60cf8> True
<keras.layers.pooling.MaxPooling2D object at 0x7f0270a09908> True
<keras.layers.convolutional.Conv2D object at 0x7f02708614a8> True
<keras.layers.normalization.BatchNormalization object at 0x7f02707a39b0> True
<keras.layers.core.Activation object at 0x7f02707a3c88> True
<keras.layers.convolutional.Conv2D object at 0x7f02707b0e80> True
<keras.layers.normalization.BatchNormalization object at 0x7f0270705b70> True
<keras.layers.core.Activation object at 0x7f027071b668> True
<keras.layers.convolutional.Conv2D object at 0x7f02706bd860> True
<keras.layers.convolutional.Conv2D obj

In [9]:
# Adding classifier on top pf Convolution model
# create model 
model = models.Sequential()

In [10]:
# Add the resnet50 convolutional model
model.add(resnet_model)

# Adding new Layers
model.add(layers.Flatten())
model.add(layers.Dense(1024, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(10, activation='softmax'))

In [11]:
# summary of model
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Model)             (None, 2, 2, 2048)        23587712  
_________________________________________________________________
flatten_1 (Flatten)          (None, 8192)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 1024)              8389632   
_________________________________________________________________
dropout_1 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 10)                10250     
Total params: 31,987,594
Trainable params: 31,934,474
Non-trainable params: 53,120
_________________________________________________________________


In [23]:
# Fitting the CNN to the images(Image Augmentation, Image Preprocessing)
from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator( 
        featurewise_center=True,
        featurewise_std_normalization=True,
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        rescale=1./255,
        shear_range=0.2,             
        zoom_range=0.2,              
        horizontal_flip=True)

In [24]:
#Normalizing the test set
test_datagen = ImageDataGenerator(rescale=1./255)

In [25]:
# setting the path to EuroSat Dataset
os.chdir('/home/sachin_sharma/Desktop/eurosat_dataset')

In [26]:
# loading training data(80%)
training_set = train_datagen.flow_from_directory(
        'TrainingSet',
         target_size=(64,64),
         batch_size=32,
         class_mode='categorical')

Found 21000 images belonging to 10 classes.


In [27]:
# loading test data(20%)
# This will create the Test set
test_set = test_datagen.flow_from_directory(
        'TestSet',
        target_size=(64,64),
        batch_size=32,
        class_mode='categorical', 
        shuffle=False)

Found 6000 images belonging to 10 classes.


In [36]:
# lets dettermine dataset characteristics
print('Training Data: ',training_set[0][0].shape)
print('Test Data: ',test_set[0][0].shape)

Training Data:  (32, 64, 64, 3)
Test Data:  (32, 64, 64, 3)


In [38]:
# now shape of single image
print('Shape of single image:', training_set[0][0][0].shape)

Shape of single image: (64, 64, 3)
